In [11]:
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns
import pandas as pd
import re
import os
import tensorflow as tf
import pickle
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import datetime
import joblib
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.models import load_model
from nltk.translate import bleu_score
import warnings
warnings.filterwarnings("ignore")

In [4]:
def final_func_1(input_sentence):
    tokenizer_sms=joblib.load('tokenizer_sms.pkl')
    tokenizer_eng=joblib.load('tokenizer_eng.pkl')
    model=load_model('model.h5')
    encoder_sms=tokenizer_sms.texts_to_sequences([input_sentence])
    encoder_pad=pad_sequences(encoder_sms,maxlen=60,padding='post',dtype='int32')
    embed=model.layers[2](encoder_pad)
    encoder_out,encoder_h,encoder_c=model.layers[4](embed)
    start_index = tokenizer_eng.word_index['<start>']
    start_index=np.reshape(start_index,(1,1))
    out=[]
    for j in range(60):
        decemb=model.layers[3](start_index)
        decoder_out,decoder_h,decoder_c=model.layers[5](decemb,[encoder_h,encoder_c])
        output=model.layers[6](decoder_out)
        encoder_h,encoder_c=decoder_h,decoder_c
        prob=np.argmax(output)
        start_index=np.reshape(prob,(1,1))
        out.append(tokenizer_eng.index_word[prob])
        if tokenizer_eng.index_word[prob]=='<end>':
            break
    return ' '.join(out)  

In [5]:
print(final_func_1('everything i think ard 180 to 200 the detail discuss wif the aprent so he interested'))

not feeling well i am still very sad i am not very sure its a new year evening i think its <end>


In [12]:
def final_func_2(input_sentence,correct):
    tokenizer_sms=joblib.load('tokenizer_sms.pkl')
    tokenizer_eng=joblib.load('tokenizer_eng.pkl')
    model=load_model('model.h5')
    encoder_sms=tokenizer_sms.texts_to_sequences([input_sentence])
    encoder_pad=pad_sequences(encoder_sms,maxlen=60,padding='post',dtype='int32')
    embed=model.layers[2](encoder_pad)
    encoder_out,encoder_h,encoder_c=model.layers[4](embed)
    start_index = tokenizer_eng.word_index['<start>']
    start_index=np.reshape(start_index,(1,1))
    out=[]
    for j in range(60):
        decemb=model.layers[3](start_index)
        decoder_out,decoder_h,decoder_c=model.layers[5](decemb,[encoder_h,encoder_c])
        output=model.layers[6](decoder_out)
        encoder_h,encoder_c=decoder_h,decoder_c
        prob=np.argmax(output)
        start_index=np.reshape(prob,(1,1))
        out.append(tokenizer_eng.index_word[prob])
        if tokenizer_eng.index_word[prob]=='<end>':
            break
    pred=' '.join(out)  
    return bleu_score.sentence_bleu(correct,pred)

In [13]:
print(final_func_2('everything i think ard 180 to 200 the detail discuss wif the aprent so he interested','everything i think around 180 to 200 the details will be discussed with the parent so is he interested <end>'))

1.2308298330855597e-231
